<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Prerequisites" data-toc-modified-id="Prerequisites-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Prerequisites</a></span></li><li><span><a href="#Imports-and-Constants" data-toc-modified-id="Imports-and-Constants-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Imports and Constants</a></span><ul class="toc-item"><li><span><a href="#Prepare-the-dataset" data-toc-modified-id="Prepare-the-dataset-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Prepare the dataset</a></span></li><li><span><a href="#Mask-out-cloud,-snow,-and-cloud-shadow" data-toc-modified-id="Mask-out-cloud,-snow,-and-cloud-shadow-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Mask out cloud, snow, and cloud shadow</a></span></li><li><span><a href="#Multi-yr-composite" data-toc-modified-id="Multi-yr-composite-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Multi-yr composite</a></span></li><li><span><a href="#Add-nightlight" data-toc-modified-id="Add-nightlight-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Add nightlight</a></span></li><li><span><a href="#Add-topography" data-toc-modified-id="Add-topography-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Add topography</a></span></li><li><span><a href="#Export-TF-Records" data-toc-modified-id="Export-TF-Records-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Export TF Records</a></span></li></ul></li></ul></div>

# Prerequisites

1. Register a Gmail email address at [https://code.earthengine.google.com](https://code.earthengine.google.come). This process may take a couple of days. Without registration, the `ee.Initialize()` command below will throw an error message.
2. Within your conda environment, run `earthengine activate` and follow the prompt. For more instructions, see [https://developers.google.com/earth-engine/python_install-conda.html](https://developers.google.com/earth-engine/python_install-conda.html).

## Instructions

This file must be run twice: once for DHS and once for LSMS.  Adjust the parameters below based on your personal Google Cloud Platform account.  Keep in mind that CSV_INPUT_PATH, GCS_FILE_PREFIX, and IS_DHS will require different values when exporting DHS data than when exporting LSMS data.

## Adjust Parameters

In [ ]:
CSV_INPUT_PATH = 'TODO'  # The path to the survey CSV, located in this repo
DHS_ASSET_ID = 'dhs_survey_data'  # The survey ID in your GEE account
GCS_BUCKET = 'sustainlab-common'  # A GCS bucket you own that will house data
GCS_FILE_PREFIX = 'Poverty_tfrecords/' # Will prefix file names, it is recommended that you use different folders for DHS and LSMS.
IS_DHS = True # False for LSMS

In [ ]:
if IS_DHS:
    file_suffix = '_dhslocs_'
else:
    file_suffix = '_lsmslocs_'

# Imports and Constants

In [ ]:
import ee

import optical_datasources as optx
import imgtools
import ee_tf_exports as tf
from ee_assets import upload_geojson_to_gee, asset_exists

In [ ]:
ee.Initialize()

## Prepare the dataset

In [ ]:
# This will upload the DHS CSV to your Google Earth Engine account
if not asset_exists(DHS_ASSET_ID):
    upload_geojson_to_gee(CSV_INPUT_PATH, DHS_ASSET_ID)

In [ ]:
dhs = ee.FeatureCollection(DHS_ASSET_ID)

In [ ]:
dhs_oldest = dhs.filter([ee.Filter.gt('year', 2008), ee.Filter.lte('year', 2011)])  # [2009-2011] inclusive
dhs_middle = dhs.filter([ee.Filter.gt('year', 2011), ee.Filter.lte('year', 2014)])  # [2012-2014] inclusive
dhs_recent = dhs.filter(ee.Filter.gt('year', 2014))  # [2015-onwards]

In [ ]:
print('Oldest size:', dhs_oldest.size().getInfo())
print('Middle size:', dhs_middle.size().getInfo())
print('Recent size:', dhs_recent.size().getInfo())

In [ ]:
countries = dhs.distinct('country').aggregate_array('country').getInfo()
display(countries)

In [ ]:
countries = dhs_oldest.distinct('country').aggregate_array('country').getInfo()
for i in countries:
    df = dhs_oldest.filter(ee.Filter.eq('country', i))
    fname = 'lx_median_2009-11_'+i+'_dhslocs'
    print(fname)
    print(df.size().getInfo())

In [ ]:
countries = dhs_middle.distinct('country').aggregate_array('country').getInfo()
for i in countries:
    df = dhs_middle.filter(ee.Filter.eq('country', i))
    fname = 'lx_median_2012-14_'+i+'_dhslocs'
    print(fname)
    print(df.size().getInfo())

In [ ]:
countries = dhs_recent.distinct('country').aggregate_array('country').getInfo()
for i in countries:
    df = dhs_recent.filter(ee.Filter.eq('country', i))
    fname = 'lx_median_2015-17_'+i+'_dhslocs'
    print(fname)
    print(df.size().getInfo())

## Mask out cloud, snow, and cloud shadow

In [ ]:
def decode_qamask(scene):
    '''
    Pixel QA Bit Flags
    Bit  Attribute
    0    Fill
    1    Clear
    2    Water
    3    Cloud Shadow
    4    Snow
    5    Cloud
    '''
    qa = scene.select('pixel_qa')
    clear = qa.bitwiseAnd(2).neq(0)
    clear = clear.updateMask(clear).rename(['pxqa_clear'])

    water = qa.bitwiseAnd(4).neq(0)
    water = water.updateMask(water).rename(['pxqa_water'])

    cloud_shadow = qa.bitwiseAnd(8).eq(0)
    cloud_shadow = cloud_shadow.updateMask(cloud_shadow).rename(['pxqa_cloudshadow'])

    snow = qa.bitwiseAnd(16).eq(0)
    snow = snow.updateMask(snow).rename(['pxqa_snow'])

    cloud = qa.bitwiseAnd(32).eq(0)
    cloud = cloud.updateMask(cloud).rename(['pxqa_cloud'])

    masks = ee.Image.cat([
        clear, water, cloud_shadow, snow,
        cloud
    ])

        # return scene.select(scene.bandNames().remove('pixel_qa')).addBands(masks)
    return masks

def mask_qaclear(img):

    clear_mask = decode_qamask(img).select('pxqa_clear')
    cloudshadow_mask = decode_qamask(img).select('pxqa_cloudshadow')
    snow_mask = decode_qamask(img).select('pxqa_snow')
    cloud_mask = decode_qamask(img).select('pxqa_cloud')
        
    return img.updateMask(cloudshadow_mask).updateMask(snow_mask).updateMask(cloud_mask).updateMask(snow_mask)

## Multi-yr composite

In [ ]:
selbands = ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2', 'TEMP1'] 

In [ ]:
roi_recent = dhs_recent.geometry()
srcoll_recent = optx.LandsatSR(roi_recent, '2015-1-1', '2017-12-31').merged
srcoll_recent = srcoll_recent.map(mask_qaclear)
srmedian_recent = srcoll_recent.select(selbands).median().reproject('EPSG:3857', None, 30)
srmedian_recent = imgtools.add_latlon(srmedian_recent)

In [ ]:
roi_middle = dhs_middle.geometry()
srcoll_middle = optx.LandsatSR(roi_middle, '2012-1-1', '2014-12-31').merged
srcoll_middle = srcoll_middle.map(mask_qaclear)
srmedian_middle = srcoll_middle.select(selbands).median().reproject('EPSG:3857', None, 30)
srmedian_middle = imgtools.add_latlon(srmedian_middle)

In [ ]:
roi_oldest = dhs_oldest.geometry()
srcoll_oldest = optx.LandsatSR(roi_oldest, '2009-1-1', '2011-12-31').merged
srcoll_oldest = srcoll_oldest.map(mask_qaclear)
srmedian_oldest = srcoll_oldest.select(selbands).median().reproject('EPSG:3857', None, 30)
srmedian_oldest = imgtools.add_latlon(srmedian_oldest)

## Add nightlight

In [ ]:
viirs = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG")
dmsp = ee.ImageCollection("NOAA/DMSP-OLS/CALIBRATED_LIGHTS_V4")

In [ ]:
nlband = ['NIGHTLIGHTS']
viirs_recent = viirs.filterDate('2015-1-1', '2017-12-31').median().select([0],nlband)
viirs_mid = viirs.filterDate('2012-1-1', '2014-12-31').median().select([0],nlband)
dmsp_oldest = dmsp.filterDate('2009-1-1', '2011-12-31').median().select([0],nlband)

In [ ]:
srmedian_recent = srmedian_recent.addBands(viirs_recent.reproject('EPSG:3857', None, 30))
srmedian_middle = srmedian_middle.addBands(viirs_mid.reproject('EPSG:3857', None, 30))
srmedian_oldest = srmedian_oldest.addBands(dmsp_oldest.reproject('EPSG:3857', None, 30))

## Add topography

In [ ]:
dem = ee.Image("USGS/SRTMGL1_003")

In [ ]:
tbands = ['ELEV','SLO', 'ASP']
topogr = ee.Algorithms.Terrain(dem).select(['elevation', 'slope', 'aspect'], tbands)

In [ ]:
srmedian_recent = srmedian_recent.addBands(topogr.reproject('EPSG:3857', None, 30))
srmedian_middle = srmedian_middle.addBands(topogr.reproject('EPSG:3857', None, 30))
srmedian_oldest = srmedian_oldest.addBands(topogr.reproject('EPSG:3857', None, 30))

## Export TF Records

In [ ]:
dhsinfo = dhs_recent.first().propertyNames().getInfo()

In [ ]:
countries = dhs_recent.distinct('country').aggregate_array('country').getInfo()
for i in countries:
    seldhs = dhs_recent.filter(ee.Filter.eq('country', i))
    fname = 'lx_median_2015-17_'+i+file_suffix
    print(fname)
    
    bands = selbands+dhsinfo+['LAT', 'LON']+tbands+nlband
    
    test = tf.get_array_patches(srmedian_recent, 30, 127, seldhs, 
                                   True, True, bands, None, 
                                   GCS_BUCKET,
                                   GCS_FILE_PREFIX, 
                                   fname)

In [ ]:
countries = dhs_middle.distinct('country').aggregate_array('country').getInfo()
for i in countries:
    seldhs = dhs_middle.filter(ee.Filter.eq('country', i))
    fname = 'lx_median_2012-14_'+i+file_suffix
    print(fname)
    
    bands = selbands+dhsinfo+['LAT', 'LON']+tbands+nlband
    
    test = tf.get_array_patches(srmedian_middle, 30, 127, seldhs, 
                                   True, True, bands, None, 
                                   GCS_BUCKET,
                                   GCS_FILE_PREFIX, 
                                   fname)

In [ ]:
countries = dhs_oldest.distinct('country').aggregate_array('country').getInfo()
for i in countries:
    seldhs = dhs_oldest.filter(ee.Filter.eq('country', i))
    fname = 'lx_median_2009-11_'+i+file_suffix
    print(fname)
    
    bands = selbands+dhsinfo+['LAT', 'LON']+tbands+nlband
    
    test = tf.get_array_patches(srmedian_oldest, 30, 127, seldhs, 
                                   True, True, bands, None, 
                                   GCS_BUCKET,
                                   GCS_FILE_PREFIX, 
                                   fname)